[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aurelio-labs/langchain-course/blob/main/chapters/07-lcel.ipynb)

#### LangChain Essentials Course

# LangChains Expression Language

LangChain is one of the most popular open source libraries for AI Engineers. It's goal is to abstract away the complexity in building AI software, provide easy-to-use building blocks, and make it easier when switching between AI service providers.

In this example, we will introduce LangChain's Expression Langauge (LCEL), abstracting a full chain and understanding how it will work. We'll provide examples for both OpenAI's `gpt-4o-mini` *and* Meta's `llama3.2` via Ollama!

In [2]:
!pip install langchain google-search-results langchain_core google-generativeai langchain_google_genai langchain_community

---

## Traditional Chains vs LCEL

In this section we're going to dive into a basic example using the traditional method for building chains before jumping into LCEL. We will build a pipeline where the user must input a specific topic, and then the LLM will look and return a report on the specified topic. Generating a _research report_ for the user.

### Traditional LLMChain

The `LLMChain` is the simplest chain originally introduced in LangChain. This chain takes a prompt, feeds it into an LLM, and _optionally_ adds an output parsing step before returning the result.

Let's see how we construct this using the traditional method, for this we need:

* `prompt` — a `PromptTemplate` that will be used to generate the prompt for the LLM.
* `llm` — the LLM we will be using to generate the output.
* `output_parser` — an optional output parser that will be used to parse the structured output of the LLM.

In [4]:
from langchain import PromptTemplate

prompt_template = "Give me a very small report on {topic}"

prompt = PromptTemplate(
    input_variables=["topic"],
    template=prompt_template
)



We will use the `gemini-2.0-flash` model with a `temperature` of `0.0`:

In [6]:
import os
from getpass import getpass
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.0,
    api_key=""
)

In [7]:
llm_out = llm.invoke("Hello there")
llm_out

AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--7c2e78c8-9dc7-47ae-8e01-45b135c7c426-0', usage_metadata={'input_tokens': 2, 'output_tokens': 10, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})

Then we define our output parser, this will be used to parse the output of the LLM. In this case, we will use the `StrOutputParser` which will parse the `AIMessage` output from our LLM into a single string.

In [8]:
from langchain.schema.output_parser import StrOutputParser

output_parser = StrOutputParser()

In [9]:
out = output_parser.invoke(llm_out)
out

'Hello! How can I help you today?'

Through the `LLMChain` class we can place each of our components into a linear `chain`.

In [10]:
from langchain.chains import LLMChain

chain = LLMChain(prompt=prompt, llm=llm, output_parser=output_parser)

/tmp/ipython-input-10-2960353250.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(prompt=prompt, llm=llm, output_parser=output_parser)


Note that the `LLMChain` _was_ deprecated in LangChain `0.1.17`, the expected way of constructing these chains today is through LCEL, which we'll cover in a moment.

We can `invoke` our `chain`, providing a `topic` that we'd like to be researched.

In [11]:
result = chain.invoke("retrieval augmented generation")
result

{'topic': 'retrieval augmented generation',
 'text': '## Retrieval Augmented Generation (RAG): A Quick Look\n\nRetrieval Augmented Generation (RAG) is a technique that enhances the capabilities of large language models (LLMs) by allowing them to access and incorporate external knowledge sources during the generation process.\n\n**How it works:**\n\n1.  **Retrieval:** Given a user query, RAG first retrieves relevant information from a knowledge base (e.g., documents, databases, web pages).\n2.  **Augmentation:** The retrieved information is then combined with the original query.\n3.  **Generation:** The LLM uses this augmented prompt to generate a more informed and contextually relevant response.\n\n**Benefits:**\n\n*   **Improved Accuracy:** Reduces hallucinations and provides more factual answers.\n*   **Access to Up-to-Date Information:** Allows LLMs to leverage current knowledge beyond their training data.\n*   **Enhanced Contextual Understanding:** Provides richer context for gener

We can view a formatted version of this output using the `Markdown` display:

In [12]:
from IPython.display import display, Markdown

display(Markdown(result["text"]))

## Retrieval Augmented Generation (RAG): A Quick Look

Retrieval Augmented Generation (RAG) is a technique that enhances the capabilities of large language models (LLMs) by allowing them to access and incorporate external knowledge sources during the generation process.

**How it works:**

1.  **Retrieval:** Given a user query, RAG first retrieves relevant information from a knowledge base (e.g., documents, databases, web pages).
2.  **Augmentation:** The retrieved information is then combined with the original query.
3.  **Generation:** The LLM uses this augmented prompt to generate a more informed and contextually relevant response.

**Benefits:**

*   **Improved Accuracy:** Reduces hallucinations and provides more factual answers.
*   **Access to Up-to-Date Information:** Allows LLMs to leverage current knowledge beyond their training data.
*   **Enhanced Contextual Understanding:** Provides richer context for generating more relevant and nuanced responses.

**In short, RAG empowers LLMs to be more knowledgeable and reliable by grounding their responses in external, verifiable information.**

That is a simple `LLMChain` using the traditional LangChain method. Now let's move onto LCEL.

## LangChain Expression Language (LCEL)

**L**ang**C**hain **E**xpression **L**anguage (LCEL) is the recommended approach to building chains in LangChain. Having superceeded the traditional methods with `LLMChain`, etc. LCEL gives us a more flexible system for building chains. The pipe operator `|` is used by LCEL to _chain_ together components. Let's see how we'd construct an `LLMChain` using LCEL.

In [13]:
lcel_chain = prompt | llm | output_parser

We can `invoke` this chain in the same way as we did before:

In [14]:
result = lcel_chain.invoke("retrieval augmented generation")
result

'## Retrieval Augmented Generation (RAG): A Quick Look\n\nRetrieval Augmented Generation (RAG) is a technique that enhances the capabilities of large language models (LLMs) by allowing them to access and incorporate external knowledge sources during the generation process.\n\n**How it works:**\n\n1.  **Retrieval:** Given a user query, RAG first retrieves relevant information from a knowledge base (e.g., documents, databases, web pages).\n2.  **Augmentation:** The retrieved information is then combined with the original query.\n3.  **Generation:** The LLM uses this augmented prompt to generate a more informed and contextually relevant response.\n\n**Benefits:**\n\n*   **Improved Accuracy:** Reduces hallucinations and provides more factual answers.\n*   **Access to Up-to-Date Information:** Allows LLMs to leverage current knowledge beyond their training data.\n*   **Enhanced Contextual Understanding:** Provides richer context for generating more relevant and nuanced responses.\n\n**In sh

The output format is slightly different, but the underlying functionality and content being output is the same. As before, we can view a formatted version of this output using the `Markdown` display:

In [15]:
display(Markdown(result))

## Retrieval Augmented Generation (RAG): A Quick Look

Retrieval Augmented Generation (RAG) is a technique that enhances the capabilities of large language models (LLMs) by allowing them to access and incorporate external knowledge sources during the generation process.

**How it works:**

1.  **Retrieval:** Given a user query, RAG first retrieves relevant information from a knowledge base (e.g., documents, databases, web pages).
2.  **Augmentation:** The retrieved information is then combined with the original query.
3.  **Generation:** The LLM uses this augmented prompt to generate a more informed and contextually relevant response.

**Benefits:**

*   **Improved Accuracy:** Reduces hallucinations and provides more factual answers.
*   **Access to Up-to-Date Information:** Allows LLMs to leverage current knowledge beyond their training data.
*   **Enhanced Contextual Understanding:** Provides richer context for generating more relevant and nuanced responses.

**In short, RAG empowers LLMs to be more knowledgeable and reliable by grounding their responses in external, verifiable information.**

### How Does the Pipe Operator Work?

Before moving onto other LCEL features, let's take a moment to understand what the pipe operator `|` is doing and _how_ it works.

Functionality wise, the pipe tells you that whatever the _left_ side outputs will be fed as input into the _right_ side. In the example of `prompt | llm | output_parser`, we see that `prompt` feeds into `llm` feeds into `output_parser`.

The pipe operator is a way of chaining together components, and is a way of saying that whatever the _left_ side outputs will be fed as input into the _right_ side.

Let's make a basic class named `Runnable` that will transform our a provided function into a _runnable_ class that we will then use with the pipe `|` operator.

In [16]:
class Runnable:
    def __init__(self, func):
        self.func = func
    def __or__(self, other):
        def chained_func(*args, **kwargs):
            return other.invoke(self.func(*args, **kwargs))
        return Runnable(chained_func)
    def invoke(self, *args, **kwargs):
        return self.func(*args, **kwargs)

With the `Runnable` class, we will be able wrap a function into the class, allowing us to then chain together multiple of these _runnable_ functions using the `__or__` method.

First, let's create a few functions that we'll chain together:

In [17]:
def add_five(x):
    return x+5

def sub_five(x):
    return x-5

def mul_five(x):
    return x*5

Now we wrap our functions with the `Runnable`:

In [20]:
add_five_runnable = Runnable(add_five)
sub_five_runnable = Runnable(sub_five)
mul_five_runnable = Runnable(mul_five)

Finally, we can chain these together using the `__or__` method from the `Runnable` class:

In [21]:
chain = (add_five_runnable).__or__(sub_five_runnable).__or__(mul_five_runnable)

chain.invoke(3)

15

So we can see that we're able to chain together our functions using `__or__`. The pipe `|` operator is simply a shortcut for the `__or__` method, so we can create the exact same chain like so:

In [22]:
chain = add_five_runnable | sub_five_runnable | mul_five_runnable

chain.invoke(3)

15

## LCEL `RunnableLambda`

The `RunnableLambda` class is LangChain's built-in method for constructing a _runnable_ object from a function. That is, it does the same thing as the custom `Runnable` class we created earlier. Let's try it out with the same functions as before.

In [23]:
from langchain_core.runnables import RunnableLambda

add_five_runnable = RunnableLambda(add_five)
sub_five_runnable = RunnableLambda(sub_five)
mul_five_runnable = RunnableLambda(mul_five)

We chain these together again with the pipe `|` operator:

In [26]:
chain = add_five_runnable | sub_five_runnable | mul_five_runnable

And call them using the `invoke` method:

In [27]:
chain.invoke(3)

15

Now we want to try something a little more testing, so this time we will generate a report, and we will try and edit that report using this functionallity.

In [30]:
prompt_str = "give me a very small report about {topic}"
prompt = PromptTemplate(
    input_variables=["topic"],
    template=prompt_str
)

In [31]:
chain = prompt | llm | output_parser

In [32]:
result = chain.invoke("AI")
display(Markdown(result))

## AI: A Quick Look

Artificial Intelligence (AI) is rapidly evolving, encompassing technologies that enable machines to perform tasks typically requiring human intelligence. From simple automation to complex problem-solving, AI is impacting various sectors like healthcare, finance, and transportation. While offering immense potential for innovation and efficiency, ethical considerations surrounding bias, job displacement, and security remain crucial areas of focus. The future of AI hinges on responsible development and deployment.

Here we are making two functions, `extract_fact` to pull out the main content of our text and `replace_word` that will replace AI with Skynet!

In [33]:
def extract_fact(x):
    if "\n\n" in x:
        return "\n".join(x.split("\n\n")[1:])
    else:
        return x

old_word = "AI"
new_word = "skynet"

def replace_word(x):
    return x.replace(old_word, new_word)

Lets wrap these functions and see what the output is!

In [34]:
extract_fact_runnable = RunnableLambda(extract_fact)
replace_word_runnable = RunnableLambda(replace_word)

In [35]:
chain = prompt | llm | output_parser | extract_fact_runnable | replace_word_runnable

In [36]:
result = chain.invoke("retrieval augmented generation")
display(Markdown(result))

Retrieval Augmented Generation (RAG) is a technique that enhances the capabilities of large language models (LLMs) by allowing them to access and incorporate external knowledge sources. Instead of relying solely on their pre-trained knowledge, RAG models first **retrieve** relevant information from a database or document store based on the user's query. This retrieved information is then **augmented** with the original query and fed into the LLM to **generate** a more informed and accurate response.
**Benefits:**
*   **Improved Accuracy:** Access to up-to-date information reduces reliance on potentially outdated or incorrect pre-trained knowledge.
*   **Reduced Hallucinations:** Grounding responses in external sources minimizes the risk of the LLM generating fabricated information.
*   **Increased Transparency:** Users can often trace the source of the information used in the response, enhancing trust and understanding.
In essence, RAG empowers LLMs to be more knowledgeable, reliable, and transparent by leveraging the power of external knowledge retrieval.